In [23]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

In [24]:
train_datagen = ImageDataGenerator(rescale = 1/255)

test_datagen = ImageDataGenerator(rescale = 1/255)

In [25]:
train_dataset = train_datagen.flow_from_directory(r'C:\Users\adi\Desktop\face detection based attendance system\divided\train',
                                                 target_size = (200, 200),
                                                 batch_size = 3,
                                                 class_mode = 'binary')
validation_dataset = test_datagen.flow_from_directory(r'C:\Users\adi\Desktop\face detection based attendance system\divided\val',
                                            target_size = (200,200),
                                            batch_size = 3,
                                            class_mode = 'binary')



Found 3200 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


In [26]:
list=validation_dataset.class_indices
lable=dict([(value,key)for key,value in list.items()])
print(lable)

{0: 'arindam', 1: 'divya'}


In [27]:
model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation="relu",input_shape=(200,200,3)),
                                  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
                                  #
                                  tf.keras.layers.Conv2D(32,(3,3),activation="relu"),
                                  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
                                  #
                                  tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
                                  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
                                  ##
                                  tf.keras.layers.Flatten(),
                                  ##
                                  tf.keras.layers.Dense(512,activation="relu"),
                                  ##
                                  tf.keras.layers.Dense(1,activation="sigmoid")
                                  
                                 ])

In [28]:
model.compile(loss="binary_crossentropy",
             optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [34]:
model_fit=model.fit(train_dataset,
                   steps_per_epoch=10,
                   epochs=6,
                    validation_data=validation_dataset
                   )

Epoch 1/6
10/10 [==============================] - 11s 1s/step - loss: 5.0749e-06 - accuracy: 1.0000 - val_loss: 0.0163 - val_accuracy: 0.9975
Epoch 2/6
10/10 [==============================] - 10s 1s/step - loss: 2.9073e-06 - accuracy: 1.0000 - val_loss: 0.0171 - val_accuracy: 0.9975
Epoch 3/6
10/10 [==============================] - 10s 1s/step - loss: 7.1163e-07 - accuracy: 1.0000 - val_loss: 0.0173 - val_accuracy: 0.9975
Epoch 4/6
10/10 [==============================] - 10s 1s/step - loss: 2.1737e-06 - accuracy: 1.0000 - val_loss: 0.0227 - val_accuracy: 0.9975
Epoch 5/6
10/10 [==============================] - 10s 1s/step - loss: 1.2821 - accuracy: 0.9000 - val_loss: 0.0962 - val_accuracy: 0.9863
Epoch 6/6
10/10 [==============================] - 10s 1s/step - loss: 0.0918 - accuracy: 0.9667 - val_loss: 0.0069 - val_accuracy: 0.9975


In [31]:
model.save("model.h5")

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 99, 99, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 64)      

In [35]:
from tensorflow import keras
import numpy as np
import cv2
from keras.models import load_model
import os
import csv
from datetime import datetime


facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
font=cv2.FONT_HERSHEY_COMPLEX


model = load_model('model.h5')


def get_className(predict_x):
    x=predict_x[0][0]
    x=int(x)
    name=lable[x]
    with open("attend.csv", "r+")as f:
        mydata=f.readlines()
        namelist=[]
        for line in mydata:
            entry=line.split(',')
            namelist.append(entry[0])
        today = datetime.now()
        time=today.strftime("%H:%M:%S")
        date=today.strftime("%d/%m/%y")
        check=name+" "+date+"\n"
        if check not in mydata:
            f.writelines(f'{name} {date}\n')
        return name
        
while True:
	sucess, imgOrignal=cap.read()
	faces = facedetect.detectMultiScale(imgOrignal,1.3,5)
	for x,y,w,h in faces:
		crop_img=imgOrignal[y:y+h,x:x+h]
		img=cv2.resize(crop_img, (200,200))
		img=img.reshape(1, 200, 200, 3)
		predict_x=model.predict(img)
        
		classes_x=np.argmax(predict_x,axis=1)
		cv2.rectangle(imgOrignal,(x,y),(x+w,y+h),(0,255,0),2)
		cv2.rectangle(imgOrignal, (x,y-40),(x+w, y), (0,255,0),-2)
		cv2.putText(imgOrignal, str(get_className(predict_x)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)
        
	cv2.imshow("Result",imgOrignal)
	k=cv2.waitKey(1)
	if k==ord('q'):
		break


cap.release()
cv2.destroyAllWindows()